In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Matplotlib created a temporary config/cache directory at C:\Users\KASB\AppData\Local\Temp\matplotlib-b1rneign because the default path (C:\Users\KASB\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
movies = pd.read_csv('MovieGenre.csv', sep=',', encoding='latin-1')

In [3]:
movies

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
11994,458522,http://www.imdb.com/title/tt458522,El cantante (2006),5.5,Biography|Drama|Music,https://images-na.ssl-images-amazon.com/images...
11995,440963,http://www.imdb.com/title/tt440963,The Bourne Ultimatum (2007),8.1,Action|Mystery|Thriller,https://images-na.ssl-images-amazon.com/images...
11996,804452,http://www.imdb.com/title/tt804452,Bratz (2007),2.8,Comedy|Family|Music,https://images-na.ssl-images-amazon.com/images...
11997,54286,http://www.imdb.com/title/tt54286,Seishun zankoku monogatari (1960),7.1,Drama,https://images-na.ssl-images-amazon.com/images...


In [4]:
movies['Poster'][11998]

'https://images-na.ssl-images-amazon.com/images/M/MV5BMTI4NTA3MjY1OF5BMl5BanBnXkFtZTcwMzM2NjA0MQ@@._V1_UY268_CR9,0,182,268_AL_.jpg'

In [5]:
movies['Genre'] = movies['Genre'].str.split('|')
movies['Genre'] = movies['Genre'].fillna("").astype('str')

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['Genre'])
tfidf_matrix.shape

(11999, 306)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.11997285, 0.08119561, 0.07597926],
       [0.11997285, 1.        , 0.        , 0.        ],
       [0.08119561, 0.        , 1.        , 0.34838397],
       [0.07597926, 0.        , 0.34838397, 1.        ]])

In [8]:
pd.to_pickle(cosine_sim,r'cosine_similarity.pickle')

In [9]:
cosine_sim = pd.read_pickle(r'cosine_similarity.pickle') 

In [10]:
titles = movies['Title']
indices = pd.Series(movies['Title'].index, index=movies['Title'])
indices

Title
Toy Story (1995)                          0
Jumanji (1995)                            1
Grumpier Old Men (1995)                   2
Waiting to Exhale (1995)                  3
Father of the Bride Part II (1995)        4
                                      ...  
El cantante (2006)                    11994
The Bourne Ultimatum (2007)           11995
Bratz (2007)                          11996
Seishun zankoku monogatari (1960)     11997
La sierra (2005)                      11998
Length: 11999, dtype: int64

In [11]:
def genre_recommendations(title):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:11]
  movie_indices = [i[0] for i in sim_scores]
  return movie_indices

In [12]:
recommendlist = genre_recommendations('The AristoCats (1970)')
print(recommendlist)

[234, 551, 580, 608, 661, 694, 994, 999, 1000, 1032]


In [13]:
arr = []
for i in recommendlist:
    arr.append({"id": i, "name": movies['Title'][i], "rating": movies['IMDB Score'][i], "poster": movies['Poster'][i], "genre": movies['Genre'][i]})

In [14]:
print(arr)

[{'id': 234, 'name': 'A Goofy Movie (1995)', 'rating': 6.8, 'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BZDczYjU4MDMtY2RjMy00Y2UyLWE2MmMtZjliYWUwNzQ3YjExXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg', 'genre': "['Animation', 'Adventure', 'Comedy']"}, {'id': 551, 'name': 'The Pagemaster (1994)', 'rating': 6.0, 'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BZjY1NjRjZjEtYjNhNS00YzY5LThjMGQtYmE0OTE3MGFiM2MyXkEyXkFqcGdeQXVyNTUyMzE4Mzg@._V1_UX182_CR0,0,182,268_AL_.jpg', 'genre': "['Animation', 'Adventure', 'Comedy']"}, {'id': 580, 'name': 'Aladdin (1992)', 'rating': 8.0, 'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BNTI4ODQ1MzAzNV5BMl5BanBnXkFtZTgwMzU4NzMxMDE@._V1_UY268_CR9,0,182,268_AL_.jpg', 'genre': "['Animation', 'Adventure', 'Comedy']"}, {'id': 608, 'name': 'The AristoCats (1970)', 'rating': 7.1, 'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMjA4OTg4MDQ4NF5BMl5BanBnXkFtZTgwNjY0MzcxMTE@._V1_UX182_CR0,0